# Access FVCOM model output from THREDDS Server

This notebook is help users get started using FVCOM output from the [GLERL THREDDS Server](https://www.glerl.noaa.gov/thredds/catalog/catalog.html). This example includes data from the Great Lakes Coastal Forecasting System ([GLCFS](https://www.glerl.noaa.gov/res/glcfs/)). Learn more about [GLCFS here](https://www.glerl.noaa.gov/res/Programs/ipemf/GLCFS_nextgen.html) and how to access both experimental and operational data on the [Data Access page here](https://www.glerl.noaa.gov/res/Programs/ipemf/glcfs_data_access.html).

The Python modules used in this example are fairly common. We also provide examples using the module [PyFVCOM](https://github.com/pmlmodelling/pyfvcom).  More examples of PyFVCOM can be found [here](https://notebook.community/pwcazenave/PyFVCOM/examples/pyfvcom_plot_example).

Thank you those whose code we based this notebook from including Rich Signell USGS [avavilable here](https://nbviewer.org/github/rsignell-usgs/ipython-notebooks/blob/master/files/FVCOM_depth_and_velocity.ipynb) and Tristan Salles [available here](https://tristansalles.github.io/Coast/queryocean/oceanforecast.html).

In [ ]:
%matplotlib inline
from pylab import *
import numpy as np
import matplotlib.tri as Tri
import matplotlib.pyplot as plt
import netCDF4
import datetime as dt
import pandas as pd
from io import StringIO


%config InlineBackend.figure_format = 'png'
plt.rcParams['mathtext.fontset'] = 'cm'

import warnings
warnings.filterwarnings('ignore')

## Loading the FVCOM file


In [ ]:
# Set the URL
url = 'https://www.glerl.noaa.gov/thredds/dodsC/glcfs/erie/nowcast/041712_0001.nc'

# Load it via OPeNDAP
nc = netCDF4.Dataset(url)

# Query the variables
for var in nc.variables.keys() :
    print(var)

In [ ]:
# take a look at the "metadata" for the variable "u"
print (nc.variables['u'])

## Set FVCOM simulation time

In [ ]:
# Enter your specific date & time in UTC
# This must correspond to the file you selected in the 'URL'
# variable above
start = dt.datetime(2023,4,17,12,0,0) # year,month,day,hour,minute,second

# Get desired time step  
time_var = nc.variables['time']
itime = netCDF4.date2index(start,time_var,select='nearest') 
print(itime, start)

In [ ]:
# Convert datetime format
dtime = netCDF4.num2date(time_var[itime],time_var.units)
daystr = dtime.strftime('%Y-%b-%d %H:%M')
print(daystr)

## Get specific data from FVCOM outputs

In [ ]:
# Get lon,lat coordinates for nodes (depth)
lat = nc.variables['lat'][:]
lon = nc.variables['lon'][:]

# Get lon,lat coordinates for cell centers (depth)
latc = nc.variables['latc'][:]
lonc = nc.variables['lonc'][:]

# Get depth
h = nc.variables['h'][:]  # depth

In [ ]:
# Get Connectivity array
nv = nc.variables['nv'][:].T - 1 

In [ ]:
# Take FVCOM Delaunay grid
tri = Tri.Triangulation(lon,lat,triangles=nv)

## Find FVCOM velocity field

In [ ]:
# Get current at layer [0 = surface, -1 = bottom]
ilayer = 0
u = nc.variables['u'][itime, ilayer, :]
v = nc.variables['v'][itime, ilayer, :]

## Visualize FVCOM forecast model

In [ ]:
# Region to plot
# print(np.min(latc), np.max(latc))
# print(np.min(lonc), np.max(lonc))
ax = [np.min(lonc), np.max(lonc), np.min(latc), np.max(latc)]

# Find velocity points in bounding box
ind = np.argwhere((lonc >= ax[0]) & (lonc <= ax[1]) & (latc >= ax[2]) & (latc <= ax[3]))

In [ ]:
# Depth contours to plot
max_depth = -int(max(nc.variables['h'][:])) - 1
levels=np.arange(max_depth,10,1)   

In [ ]:
# To make the figure readable subsample the number of vector to draw.
subsample = 3
np.random.shuffle(ind)
Nvec = int(len(ind) / subsample)
idv = ind[:Nvec]

## Plot in iPython

In [ ]:
# tricontourf plot of water depth with vectors on top
fig1 = plt.figure(figsize=(10,7))

ax1 = fig1.add_subplot(aspect=(1.0/np.cos(np.mean(lat)*np.pi/180.0)))

# Water depth
plt.tricontourf(tri, -h, levels=levels, cmap=plt.cm.ocean)
plt.axis(ax)

ax1.patch.set_facecolor('0.5')
cbar=plt.colorbar()
cbar.set_label('Water Depth (m)', rotation=-90, labelpad=18)

# Quiver plot
Q = ax1.quiver(lonc[idv],latc[idv],u[idv],v[idv],scale=20)
qk = plt.quiverkey(Q,0.92,0.08,0.50,'0.5 m/s',labelpos='W')

plt.title('Lake Erie Coastal Forecasting System Velocity, Layer %d, %s' % (ilayer, daystr))
plt.show()

## Extract Temperature Profile

In [ ]:
# Enter desired (Station, Lat, Lon) values here:
x = '''
Station, Lat, Lon
Cleveland OH,   41.72883, -81.798497
'''
# Create a Pandas DataFrame
obs=pd.read_csv(StringIO(x.strip()), sep=",\s*",index_col='Station',engine='python')
# Convert longitude coordinate
obs['Lon'] = 360.0 + obs['Lon']
obs

In [ ]:
# Find the indices of the points in (x,y) closest to the points in (xi,yi)
def nearxy(x,y,xi,yi):
    
    ind=np.ones(len(xi),dtype=int)
    for i in np.arange(len(xi)):
        dist=np.sqrt((x-xi[i])**2+(y-yi[i])**2)
        ind[i]=dist.argmin()
        
    return ind

# Query to find closest NECOFS nodes to station locations
obs['NODE-ID'] = nearxy(nc['lon'][:],nc['lat'][:],obs['Lon'],obs['Lat'])
obs

In [ ]:
# Get temperature profile from location named above
# At the time defined above
nsta=len(obs)
z=np.ones((len(nc.variables['siglay'][:,0]),nsta))

for i in range(nsta):
    # Find depth in meters of center of sigma layer
    depths=nc.variables['siglay'][:,obs['NODE-ID'][i]] * nc.variables['h'][obs['NODE-ID'][i]]
    z[:,i] = nc['temp'][0,:,obs['NODE-ID'][i]]
    
    
# Make a DataFrame out of the interpolated time series at each location
zvals=pd.DataFrame(z[:],index=depths)
zvals.index.name = 'depth_m'
zvals.columns=['temp_C']

# Or print all values
print(zvals)

In [ ]:
# Plot temperature profile

fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
ax1.plot(zvals['temp_C'],zvals.index,'o-')

# Draw x label
ax1.set_xlabel('Temperature (C)')
ax1.xaxis.set_label_position('top') # this moves the label to the top
ax1.xaxis.set_ticks_position('top') # this moves the ticks to the top

# Draw y label
ax1.set_ylabel('Depth (m)')

plt.show()

## Extract Current Profile

In [ ]:
# Enter desired (Station, Lat, Lon) values here:
x = '''
Station, Lat, Lon
Cleveland OH,   41.72883, -81.798497
'''
# Create a Pandas DataFrame
obs=pd.read_csv(StringIO(x.strip()), sep=",\s*",index_col='Station',engine='python')
# Convert longitude coordinate
obs['Lon'] = 360.0 + obs['Lon']
obs


In [ ]:
# Same code as the temperature profile above with addition of 
# nearest cell center coordinate (e.g., 'latc' & 'lonc')
# Find the indices of the points in (x,y) closest to the points in (xi,yi)
def nearxy(x,y,xi,yi):
    
    ind=np.ones(len(xi),dtype=int)
    for i in np.arange(len(xi)):
        dist=np.sqrt((x-xi[i])**2+(y-yi[i])**2)
        ind[i]=dist.argmin()
        
    return ind

# Query to find closest NECOFS nodes to station locations
obs['NODE-ID'] = nearxy(nc['lon'][:],nc['lat'][:],obs['Lon'],obs['Lat'])
obs['NODE-ID-c'] = nearxy(nc['lonc'][:],nc['latc'][:],obs['Lon'],obs['Lat'])
obs

In [ ]:
# Create empty lists to store u,v value information
nsta=len(obs)
sigma_layers = len(nc.variables['siglay'][:,0]) # number of depth layers
ui=np.ones((sigma_layers,nsta))
vi=np.ones((sigma_layers,nsta))

In [ ]:
# Get u values profile from location named above
# At the time defined above
for i in range(nsta):
    ui[:,i] = nc['u'][itime,:,obs['NODE-ID-c'][i]]

In [ ]:
# Get v values profile from location named above
# At the time defined above
for i in range(nsta):
    vi[:,i] = nc['v'][itime,:,obs['NODE-ID-c'][i]]

In [ ]:
# Get depths nearest observation point
for i in range(nsta):
    depths=nc.variables['siglay'][:,obs['NODE-ID'][i]] * nc.variables['h'][obs['NODE-ID'][i]]

# Make a DataFrame out of the interpolated time series at each location
uvals=pd.DataFrame(ui[:],index=depths)
uvals.index.name = 'depth_m'
uvals.columns=['u']

vvals=pd.DataFrame(vi[:],index=depths)
vvals.index.name = 'depth_m'
vvals.columns=['v']

circ_profile = pd.concat([uvals, vvals], axis=1)

#Print all values
print(circ_profile)